# Snapchat Political Ads
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the reach (number of views) of an ad.
    * Predict how much was spent on an ad.
    * Predict the target group of an ad. (For example, predict the target gender.)
    * Predict the (type of) organization/advertiser behind an ad.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings


### Introduction
TODO

### Baseline Model
TODO

### Final Model
TODO

### Fairness Evaluation
TODO

# Code

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

In [61]:
eighteen = pd.read_csv("2018.csv")
nineteen = pd.read_csv("2019.csv")
result = pd.concat([eighteen, nineteen])
result.columns

Index(['ADID', 'CreativeUrl', 'Spend', 'Impressions', 'StartDate', 'EndDate',
       'OrganizationName', 'BillingAddress', 'CandidateBallotInformation',
       'PayingAdvertiserName', 'Gender', 'AgeBracket', 'CountryCode',
       'RegionID', 'ElectoralDistrictID', 'LatLongRad', 'MetroID', 'Interests',
       'OsType', 'Segments', 'LocationType', 'Language',
       'AdvancedDemographics', 'Targeting Connection Type',
       'Targeting Carrier (ISP)', 'Targeting Geo - Postal Code',
       'CreativeProperties'],
      dtype='object')

Prediction: We will try to predict the reach, or the number of views, of an ad. This is a regression problem. 

### Baseline Model

In [35]:
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from datetime import datetime

In [70]:
# changing all dates to datetime objects
result['StartDate'] = result['StartDate'].str[:-1]
result['EndDate'] = result['EndDate'].str[:-1]
result['StartDate'] = pd.to_datetime(result['StartDate'])
result['EndDate'] = pd.to_datetime(result['EndDate'], errors='coerce')


KeyError: 'StartDate'

In [71]:
# EndDate                         fill with "Present" or current time
result['EndDate'] = result['EndDate'].fillna(datetime.utcnow())
# Gender                          fill with "All"
result['Gender'] = result['Gender'].fillna("All")
# display(result[['StartDate', 'EndDate']])
result['Duration'] = abs(result['EndDate']- result['StartDate'])
result ["Duration"] = result['Duration'].apply(lambda x: x.seconds)
display(result['Duration'])
# CandidateBallotInformation      ??? type of ballot information
# CreativeProperties              ??? url attachments
# Interests                       ??? agonstic (not specified?)
# LocationType                    ??? specfic location targetting, like airports, education, college, etc
# Language                        ??? agonstic (not specified?)
# AdvancedDemographics            ??? agonstic (not specified?)
#LocationType, CreativeProperties, CandidateBallotInformation
# Quantitative: Impressions, Spend, Duration
# Nominal: CountryCode, Gender, 

KeyError: 'EndDate'

In [68]:
result = result[['Impressions', 'Duration', 'Spend','Gender', 'CountryCode']]

In [53]:
X = result.drop(['Impressions'], axis=1)
y = result.Impressions

In [54]:
# TODO
types = X.dtypes
catcols = types.loc[types == np.object].index
numcols = types.loc[types != np.object].index

cats = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='NULL')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
])

ct = ColumnTransformer([
    ('catcols', cats, catcols),
    ('numcols', SimpleImputer(strategy='constant', fill_value=0), numcols)
])

pl = Pipeline([('feats', ct), ('reg', LinearRegression())])

X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
pl.fit(X_tr, y_tr)
rsquared = pl.score(X_ts, y_ts)
print("r-squared: " + str(rsquared))
preds = pl.predict(X_ts)
rmse = np.sqrt(np.mean(preds - y_ts)**2)
print("rmse: " + str(rmse))

TypeError: float() argument must be a string or a number, not 'Timedelta'

### Final Model

In [ ]:
# TODO

### Fairness Evaluation

In [ ]:
# TODO